## Downlaod Data From GEE

In [2]:
import ee
ee.Authenticate()

Enter verification code:  4/1AX4XfWg8BUSWCPW6OsImngrd9s3-wKM8gTGM4OYjhEcDBynU9adHJicpm6I



Successfully saved authorization token.


In [10]:
ee.Initialize()

In [11]:
from download_utils import Mapdisplay
import folium
import geehydro
import time 
import pandas as pd
import numpy as np

In [12]:
def _get_test_image():
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')
    countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");
    Country = countries.filter(ee.Filter.eq("country_co","BF")); #try the bahamas
    
    landsat = l8.filterDate('2019-01-01', '2019-10-31') \
                .filterBounds(Country) 
    
    ## this is a cloud masking method in EE
    composite = ee.Algorithms.Landsat.simpleComposite(**{
      'collection': landsat,
      'asFloat': True
    })
    
    return composite

In [13]:
c = _get_test_image()
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");
Country = countries.filter(ee.Filter.eq("country_co","BF"))
rgbVis = {'bands': ["B4", "B3", "B2"], 'min':0, 'max': 0.3}
Map = folium.Map(location=[40, -100], zoom_start=4)
#Map.setOptions('SATELLITE')
Map.addLayer(c, rgbVis, "RGB")
Map.centerObject(Country, 4)
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

In [15]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': c.toFloat(),
    'description': 'test',
    'folder':'GEE_Resources_Raw',   ## it will create a folder an dump the image there 
    #'scale': scale,
    #'crs': projection['crs'],
    #'crsTransform': projection['transform'],
    'region': Country.geometry(),
    'fileFormat': 'GeoTIFF',
    'maxPixels':1e13,
})
task.start()

while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(20)
    
print(task.status())

Polling for task (id: EK34GDSOFOJHUXLN7DAW6TS5).
Polling for task (id: EK34GDSOFOJHUXLN7DAW6TS5).
Polling for task (id: EK34GDSOFOJHUXLN7DAW6TS5).
{'state': 'COMPLETED', 'description': 'test', 'creation_timestamp_ms': 1651803900593, 'update_timestamp_ms': 1651803955492, 'start_timestamp_ms': 1651803930317, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1IlG_KVAL6C3W9xrswbswFBRKjve2YXSu'], 'attempt': 1, 'id': 'EK34GDSOFOJHUXLN7DAW6TS5', 'name': 'projects/earthengine-legacy/operations/EK34GDSOFOJHUXLN7DAW6TS5'}


In [6]:
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");
Country = countries.filter(ee.Filter.eq("country_na","China"));

landsat = l8.filterDate('2019-01-01', '2019-10-31') \
            .filterBounds(Country) 

## this is a cloud masking method in EE
composite = ee.Algorithms.Landsat.simpleComposite(**{
  'collection': landsat,
  'asFloat': True
})
## add layer info
rgbVis = {'bands': ["B4", "B3", "B2"], 'min':0, 'max': 0.3}

In [7]:
Map = folium.Map(location=[40, -100], zoom_start=4)
#Map.setOptions('SATELLITE')
Map.addLayer(composite, rgbVis, "RGB")
Map.centerObject(Country, 4)
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

### Download One Country From L8

In [73]:
projection = composite.select('B2').projection().getInfo();

In [74]:
projection

{'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0], 'type': 'Projection'}

In [75]:
f_name = "L8_Country"
scale = 30 ## usually 30 

In [31]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': composite.toFloat(),
    'description': f_name,
    'folder':'GEE_Resources_Raw',   ## it will create a folder an dump the image there 
    'scale': scale,
    'crs': projection['crs'],
    'crsTransform': projection['transform'],
    'region': Japan.geometry(),
    'fileFormat': 'GeoTIFF',
    'maxPixels':1e13,
})
task.start()

while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(1200)
    
print(task.status())

NameError: name 'composite' is not defined

### Download From Modise

In [4]:
locations_filepath="data/yield_data.csv"
collection_id="MODIS/061/MOD09A1"

In [5]:
imgcoll = ee.ImageCollection(collection_id) \
    .filterBounds(ee.Geometry.Rectangle(-106.5, 50, -64, 23)) \
    .filterDate("2002-12-31", "2003-8-4")

In [6]:
# Get the number of images.
count = imgcoll.size()
print('Count: ', str(count.getInfo())+'\n')

Count:  27



In [7]:
def _append_im_band(current, previous):
    # Transforms an Image Collection with 1 band per Image into a single Image with items as bands
    # Author: Jamie Vleeshouwer

    # Rename the band
    previous = ee.Image(previous)
    current = current.select([0, 1, 2, 3, 4, 5, 6])
    # Append it to the result (Note: only return current item on first element/iteration)
    return ee.Algorithms.If(
        ee.Algorithms.IsEqual(previous, None),
        current,
        previous.addBands(ee.Image(current)),
    )

In [8]:
img = imgcoll.iterate(_append_im_band)
img = ee.Image(img)

In [9]:
print(len(img.getInfo()['bands']))
print(count.getInfo()*7)

189
189


In [10]:
region = ee.FeatureCollection("TIGER/2018/Counties")

In [11]:
# turn the strings into numbers, see
# https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_Counties
def county_to_int(feature):
    return feature.set("COUNTYFP", ee.Number.parse(feature.get("COUNTYFP")))

def state_to_int(feature):
    return feature.set("STATEFP", ee.Number.parse(feature.get("STATEFP")))

region = region.map(county_to_int)
region = region.map(state_to_int)

In [12]:
df = pd.read_csv(locations_filepath)
#df.head()

In [13]:
x = np.unique(df[["State ANSI", "County ANSI"]].values, axis=0)
x.shape

(3522, 2)

In [14]:
state_id, county_id = x[10]

In [15]:
fname = "{}_{}".format(int(state_id), int(county_id))

In [16]:
fname

'1_35'

In [17]:
file_region = region.filterMetadata(
    "COUNTYFP", "equals", int(county_id)
).filterMetadata("STATEFP", "equals", int(state_id))

In [18]:
file_region = ee.Feature(file_region.first())
processed_img = img.clip(file_region)
#file_region = None

In [36]:
folder_name = 'GEE_Resources_Raw'
prefix = 'county1'
print(fname)
coordinate_system="EPSG:4326",
scale=500

1_35


In [44]:
projection = processed_img.select(['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03']).projection().getInfo()
projection

{'type': 'Projection',
 'crs': 'SR-ORG:6974',
 'transform': [463.3127165279165,
  0,
  -20015109.354,
  0,
  -463.3127165279167,
  7783653.637669001]}

In [47]:
test_crs = 'EPSG:4326'

In [49]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': processed_img.toFloat(),
    'description': fname,
    'folder':folder_name,   ## it will create a folder an dump the image there 
    #'scale': scale,
    'fileNamePrefix':"{}/{}_crs2".format(prefix,fname),
    #'crs': test_crs, #projection['crs'],
    #'crsTransform': projection['transform'],
    'region': file_region.geometry(),
    'fileFormat': 'GeoTIFF',
    'maxPixels':1e13,
})
task.start()

while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(120)
    
print(task.status())

Polling for task (id: ULJ3YIARCPCH4WC2337V4RFO).
{'state': 'COMPLETED', 'description': '1_35', 'creation_timestamp_ms': 1651629923717, 'update_timestamp_ms': 1651630008364, 'start_timestamp_ms': 1651629960283, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1IlG_KVAL6C3W9xrswbswFBRKjve2YXSu'], 'attempt': 1, 'id': 'ULJ3YIARCPCH4WC2337V4RFO', 'name': 'projects/earthengine-legacy/operations/ULJ3YIARCPCH4WC2337V4RFO'}


In [22]:
dataset = ee.FeatureCollection('TIGER/2018/Counties');
visParams = {
  'palette': ['purple', 'blue', 'green', 'yellow', 'orange', 'red'],
  'min': 0,
  'max': 50,
  'opacity': 0.8,
}

## Turn the strings into numbers
dataset = dataset.map(lambda f: f.set('STATEFP', ee.Number.parse(f.get('STATEFP'))))
dataset = dataset.map(lambda f: f.set('COUNTYFP', ee.Number.parse(f.get('COUNTYFP'))))


file_region = dataset.filterMetadata("STATEFP", "equals", int(state_id)) \
            .filterMetadata(
                "COUNTYFP", "equals", int(county_id)
            )

image = ee.Image().float().paint(file_region)
                      
countyOutlines = ee.Image().float().paint(**{
  'featureCollection': dataset,
  'color': 'black',
  'width': 1
});


Map = folium.Map(location=[40, -100], zoom_start=4)
#Map.setOptions('SATELLITE')
#Map.setCenter(-99.844, 37.649, 5)
Map.centerObject(file_region, 8)
Map.addLayer(image, visParams, 'TIGER/2018/Counties')
Map.addLayer(countyOutlines, {}, 'county outlines')
#Map.addLayer(dataset, {}, 'for Inspector', false)
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

In [29]:
one_img = imgcoll.mean().clipToCollection(file_region).select(['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03'])


In [30]:
Map = folium.Map(location=[40, -100], zoom_start=4)
#Map.setOptions('SATELLITE')
#Map.setCenter(-99.844, 37.649, 5)
Map.centerObject(file_region, 8)
Map.addLayer(one_img, {
  'min': -100.0,
  'max': 3000.0,
  #'gamma': 1.4,
}, 'TIGER/2018/Counties')
Map.addLayer(countyOutlines, {}, 'county outlines')
#Map.addLayer(dataset, {}, 'for Inspector', false)
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map